In [ ]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
%matplotlib widget
matplotlib.rc('font', size=18)
import uncertainties as unc
import scipy.special as ss

from tqdm.notebook import tqdm

import sys
sys.path.append('../')
import general_purpose.utilities as ut
import general_purpose.uplotlib as uplt

from importlib import reload

import rea.reconstruct as rec

In [ ]:
def get_run(folder):
    run = {'rec_dict': ut.json2dict(f'{folder}/reconstructed.json')}
    
    run['observables'] = {}
    for rname, r in run['rec_dict']['members'].items():
        last_segment_traj = np.load(f"{folder}/{run['rec_dict']['folders'][-1]}/{r['ancestry'][-1]}-traj.npy")
        run['observables'][rname] = last_segment_traj[-1,1]
        
    return run

def expectation(run, func, **kwargs):
    _f = np.array([func(x, **kwargs) for x in run['observables'].values()])
    _w = np.array([r['weight'] for r in run['rec_dict']['members'].values()])
    # _w = np.exp(-np.array([r['cum_log_escore'] for r in rec_dict['members'].values()]))
    # _n = np.exp(rec_dict['cum_log_norm_factor'])

    mean = np.mean(_f*_w)
    std = np.sqrt(np.mean((_f*_w)**2)/len(_w))

    return unc.ufloat(mean,std)

In [ ]:
def f(x, a):
    return float(x>a)

lam = 1
sig = 1
def ou_var(t):
    return sig**2/(2*lam)*(1 - np.exp(-2*lam*t))

cum_gaus = lambda x: 0.5*(1 + ss.erf(x/np.sqrt(2)))

overcoming_prob = lambda x,v: cum_gaus(-x/np.sqrt(v))

v = ou_var(2)

In [ ]:
c_runs = [get_run(f'./__test__/c{i}--k__0--nens__999--T__10/') for i in range(6)]
# c_runs = [get_run(f'./__test__/c{i}--k__0--nens__100--T__10/') for i in range(6)]
# runs = [get_run(f'./__test__/f{i}--k__4--nens__999--T__10') for i in range(6)]
runs = [get_run(f'./__test__/f{i}--k__2--nens__999--T__10') for i in range(8)]
# runs = [get_run(f'./__test__/f{i}--k__4--nens__100--T__10') for i in range(14)]

In [ ]:
a = np.linspace(0,4,101)

es = [np.array([expectation(run, f, a=_a) for _a in a]) for run in runs]
c_es = [np.array([expectation(run, f, a=_a) for _a in a]) for run in c_runs]
e_th = overcoming_prob(a,v)

In [ ]:
n_es = np.vstack([uplt.nominal_value(e) for e in es])
c_n_es = np.vstack([uplt.nominal_value(e) for e in c_es])

s_es = np.vstack([uplt.std_dev(e) for e in es])
c_s_es = np.vstack([uplt.std_dev(e) for e in c_es])

m_es = uplt.avg(n_es, axis=0)
c_m_es = uplt.avg(c_n_es, axis=0)

relative_error = np.sqrt(np.mean((n_es/uplt.nominal_value(m_es) - 1)**2, axis=0))
c_relative_error = np.sqrt(np.mean((c_n_es/uplt.nominal_value(c_m_es) - 1)**2, axis=0))

est_rel_err = uplt.avg(s_es/n_es, axis=0)
c_est_rel_err = uplt.avg(c_s_es/c_n_es, axis=0)

ms_bias = np.sqrt(np.mean((n_es/e_th - 1)**2, axis=0))
c_ms_bias = np.sqrt(np.mean((c_n_es/e_th - 1)**2, axis=0))

In [ ]:
plt.close(2)
fig,ax = plt.subplots(num=2, figsize=(9,6))

plt.semilogy(a, e_th, color='black', label='theoretical', linestyle='dashed')
uplt.errorband(a, m_es, color='red', label='estimated')
uplt.errorband(a, c_m_es, alpha=0.5, color='blue', label=r'control: $k=0$')

# for e in es:
#     uplt.plot(a,e, alpha=0.5)
# plt.plot(a,uplt.std_dev(e)/uplt.nominal_value(e))

plt.xlabel(r'$a$')
plt.ylabel(r'$\mathbb{P}\left[ x(t) > a|x(0) = 0 \right]$')

plt.title(r'$N = 1000,\, k=2$')

plt.legend()

fig.tight_layout()

In [ ]:
plt.close(3)
fig,ax = plt.subplots(num=3, figsize=(9,6))

plt.plot(a, relative_error, color='red', label='$k=2$') # error between different realization of the algorithm
plt.plot(a, ms_bias, color='red', linestyle='dashed') # mean square bias

plt.plot(a, c_relative_error, color='blue', label='$k=0$') # error between different realization of the algorithm
plt.plot(a, c_ms_bias, color='blue', linestyle='dashed') # mean square bias

# for e in es:
#     plt.plot(a, uplt.std_dev(e)/uplt.nominal_value(e)) # error estimated by each realization

plt.legend()
plt.xlabel('$a$')
plt.ylabel('relative error')
fig.tight_layout()

In [ ]:
plt.close(4)
fig,ax = plt.subplots(num=4, figsize=(9,6))

uplt.plot(a, m_es)
plt.plot(a, e_th, color='black')

fig.tight_layout()

In [ ]:
plt.close(5)
fig,ax = plt.subplots(num=5, figsize=(9,6))

plt.plot(a, relative_error, color='red', label='$k=2$') # error between different realization of the algorithm
plt.plot(a, ms_bias, color='red', linestyle='dashed') # mean square bias
uplt.errorband(a, est_rel_err, color='red', linestyle='dotted')

plt.plot(a, c_relative_error, color='blue', label='$k=0$') # error between different realization of the algorithm
plt.plot(a, c_ms_bias, color='blue', linestyle='dashed') # mean square bias
uplt.errorband(a, c_est_rel_err, color='blue', linestyle='dotted')

# for e in es:
#     plt.plot(a, uplt.std_dev(e)/uplt.nominal_value(e)) # error estimated by each realization

plt.ylim(0,1)
plt.legend()
plt.xlabel('$a$')
plt.ylabel('relative error')
fig.tight_layout()

In [ ]:
np.nanmean(relative_error/est_rel_err)